In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from joblib import dump,load

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
df

,Formula,K_frac,Rb_frac,Cs_frac,MA_frac,FA_frac,Li_frac,Ca_frac,Sr_frac,Ba_frac,...,X_Heat_of_fusion(kJ/mol),A_Heat_of_vap(kJ/mol),B_Heat_of_vap(kJ/mol),X_Heat_of_vap(kJ/mol),A_Electronegativity,B_Electronegativity,X_Electronegativity,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,MAGeCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,334.30,10.20,2.55,2.01,3.16,-0.391,1.954,0.148
1,MAGeBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,334.30,14.73,2.55,2.01,2.96,-0.243,1.612,0.204
2,MAGeI_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7.76,23.85,334.30,20.90,2.55,2.01,2.66,-0.030,1.311,0.231
3,MASnCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,290.37,10.20,2.55,1.96,3.16,-0.246,1.582,0.165
4,MASnBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,290.37,14.73,2.55,1.96,2.96,-0.228,1.262,0.198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,FAGeI_1.125_Cl_1.875,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.92,29.00,334.30,14.21,2.57,2.01,2.97,-1.075,2.156,0.110
484,FASnI_0.375_Cl_2.625,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.78,29.00,290.37,11.54,2.57,1.96,3.10,-1.133,2.576,0.062
485,MAPbCl_1.125_I_1.875,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,6.05,23.85,177.90,16.89,2.55,2.33,2.85,-0.003,1.867,0.164
486,MASrCl_0.75_I_2.25,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,6.62,23.85,136.90,18.23,2.55,0.95,2.79,0.101,3.693,0.001


In [3]:
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
Y

,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,-0.391,1.954,0.148
1,-0.243,1.612,0.204
2,-0.030,1.311,0.231
3,-0.246,1.582,0.165
4,-0.228,1.262,0.198
...,...,...,...
483,-1.075,2.156,0.110
484,-1.133,2.576,0.062
485,-0.003,1.867,0.164
486,0.101,3.693,0.001


In [4]:
Y=Y['Decomposition Energy (eV p.f.u.)']
Y

0     -0.391
1     -0.243
2     -0.030
3     -0.246
4     -0.228
       ...  
483   -1.075
484   -1.133
485   -0.003
486    0.101
487   -0.689
Name: Decomposition Energy (eV p.f.u.), Length: 488, dtype: float64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 57) ; y_train's shape is (390,)
X_test's shape is (98, 57) ; y_test's shape is (98,)


In [6]:
##gridsearch and cv select model##
param_grid={ 'learning_rate': [0.01,0.05,0.1,0.15],
              'n_estimators': [15,50,100,200],
              'max_depth': [4,5,6,7,8],
              'min_child_weight': [3,4,6,7],
              'subsample': [0.7,0.8,0.9,1],
              'colsample_bytree': [0.3,0.5,0.6,0.7],
              'gamma': [0,0.1,0.2,0.3,0.4],
                'reg_alpha': [0,0.1,0.2,0.3],
                'reg_lambda': [1,2,5]
            }

In [7]:
xgbmodel = xgb.sklearn.XGBRegressor()
model = GridSearchCV(estimator =xgbmodel,scoring='neg_root_mean_squared_error', param_grid = param_grid,cv=3, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train_stand, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('XGB8cBM1117.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 3 folds for each of 307200 candidates, totalling 921600 fits


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0            0.017982      0.002996         0.003853        0.001316   
1            0.018394      0.002882         0.004195        0.000642   
2            0.015066      0.002494         0.002671        0.000474   
3            0.016404      0.003910         0.002333        0.000470   
4            0.016069      0.002030         0.003186        0.000632   
...               ...           ...              ...             ...   
307195       0.168578      0.024719         0.001002        0.000003   
307196       0.180032      0.037333         0.001333        0.000471   
307197       0.180027      0.040042         0.001332        0.000470   
307198       0.178685      0.049054         0.001667        0.000472   
307199       0.155077      0.014812         0.001333        0.000471   

       param_colsample_bytree param_gamma param_learning_rate param_max_depth  \
0                         0.3           0             